In [1]:
import numpy as np
import sympy as sym


In [18]:
s,t = sym.symbols('s t')
x1,x2,x3 = sym.symbols('x1 x2 x3')


In [3]:

A=sym.Matrix([[3, 10],[5,2]])
B=sym.Matrix([1,1])
C=sym.Matrix([1,0])
D=0
def ss2tf(A,B,C,D):
    TF=C.transpose() * (sym.eye(A.shape[0])*s-A).inv() * B + sym.Matrix([D])
    TF=sym.simplify(TF)
    return TF

ss2tf(A,B,C,D)

Matrix([[(s + 8)/(s**2 - 5*s - 44)]])

In [4]:
A=sym.Matrix([[0, 1,0],[0,0,1],[-6,-11,-6]])

def isStable(A):
    for i in A.eigenvals().keys():
        if i>0:
            return "unstable"
        else:
            return "stable"
isStable(A)

'stable'

In [5]:
A=sym.Matrix([[0, 1],[-1,-2]])
C=sym.Matrix([1,0])

def isObservable(A,C):
    n=A.shape[0]        
    O=C.transpose()
    for i in range(n-1):
        O=O.row_insert(i+1,C.transpose()*A**(i+1))
    
    if O.rank()==n:
        return "completly observable"
    elif O.rank()>n/2:
        return "not completly observable"
    else :
        return "not observable"

isObservable(A,C)


'completly observable'

In [6]:
def isControllable():
    pass



In [7]:
def tf2ss(num,den):
    pass

In [8]:
f = 6/(s**3+6*s**2+11*s+6)
p=sym.apart(f)
str(p)

'3/(s + 3) - 6/(s + 2) + 3/(s + 1)'

In [21]:
A=sym.Matrix([[0, 1,0],[0,0,1],[-10,-17,-8]])
B=sym.Matrix([0,0,1])
x0=sym.Matrix([1,1,0])
u=1/s

def solve(A,B,x0,u):
    x=(sym.eye(A.shape[0])*s-A).inv()*x0 -(sym.eye(A.shape[0])*s-A).inv()*B
    x=sym.inverse_laplace_transform(x, s, t)
    x=sym.simplify(x)

    return x


In [22]:
solve(A,B,x0,u)

Matrix([
[ (12*exp(4*t) - 10*exp(3*t) + 1)*exp(-5*t)*Heaviside(t)/3],
[(-12*exp(4*t) + 20*exp(3*t) - 5)*exp(-5*t)*Heaviside(t)/3],
[(12*exp(4*t) - 40*exp(3*t) + 25)*exp(-5*t)*Heaviside(t)/3]])